## 이 문서를 수정할 당신에게...
#### 현재 상황은 아래와 같습니다.
1. 우리의 앱에서 중요한 요소 중 하나는 와인을 추천하고, 이를 실제로 구매할 수 있는 링크에 연결하는 것입니다. 또, 실제 와인바를 추천해주기도 합니다. 이를 위해서 실제 와인을 파는 사이트와 와인바를 데이터베이스로 만들어두어야하기 때문에 이를 위해서는 크롤링이 필요합니다.
2. 현재 와인나라와 와인앤모어에 대한 크롤링을 완료한 상태입니다. 하지만 이를 통합된 하나의 데이터베이스로 만드는 과정이 필요합니다.
- 와인나라: [winenara.json](./winenara_scrapy/winenara.json)
- 와인앤모어: [wine_and_more.json](./wine_and_more.json)
3. 와인나라에 대한 데이터를 크롤링했으나 바디, 산도 등 없는 데이터가 많았습니다. 이 때문에 X wine dataset을 이용해 부족한 부분을 추가해주어야합니다.

#### 당신의 목표는 아래와 같습니다.
1. 아래 사이트에 대한 크롤링을 진행합니다. 크롤링이 필요한 요소는 와인이나 와인바 추천에 필요하다고 생각하는 모든 것을 하면 됩니다. 또한 추가로 더 좋은 사이트의 크롤링도 좋습니다.

    <strong>와인</strong>
    - [와인나라](https://www.winenara.com/shop/main)
    - [와인앤모어](https://www.wineandmore.co.kr/)
    - [CU 편의점](https://pocketcu.co.kr/)
    - [SSG 와인장터](https://emart.ssg.com/specialStore/wineliquor/main.ssg)
    - [롯데온](https://www.lotteon.com/search/search/search.ecn?render=search&platform=pc&q=%EC%99%80%EC%9D%B8&mallId=1)


<br>

2. 이후 와인에 대한 데이터베이스를 통합합니다. 현재 아래의 schema를 생각하고 있으나 다른 것을 추가하는 것도 좋습니다.
- url(required): 해당 페이지의 url
- site_name(required): 해당 페이지의 출처(와인나라, 와인앤모어 등)
- price(required): 가격
- name: 한글 이름
- en_name: 영어 이름
- img_url(required): 이미지 url 리스트
- body: 바디감(1~5점)
- acidity: 산도(1~5점)
- tannin: 타닌(1~5점)
- sweetness: 당도(1~5점)
- alcohol: %
- wine_type(required): 와인타입(레드, 화이트 등)
- country: 원산지(미국, 독일 등)
- grape: 포도품종
- rating: vivino 점수(1~5점)
- pickup_location: 픽업장소
- vivino_link: vivino 사이트의 와인 상세페이지 url
- flavor_description: 맛과 향에대한 자연어 설명
- winery: 와인을 주조한 곳에 대한 자연어 설명
- handling: 적정온도, 디캔팅, 빈티지에 대한 설명
- paring: 푸드 페어링

#### 참고 사항
현재 작성된 크롤링은 아래의 프레임워크로 작업했습니다.
- [Scrapy](https://docs.scrapy.org/en/latest/intro/tutorial.html)
- [Selenium](https://selenium-python.readthedocs.io/)

## 와인나라 크롤링

와인나라에 있는 와인에 대한 정보를 가져오는 것을 목적으로 한다.

크롤링에는 [Scrapy](https://docs.scrapy.org/en/latest/intro/tutorial.html)를 사용한다.

scrapy를 사용하기 위해서 [새로운 프로젝트](./winenara_scrapy)에서 작업을 한다. 여기서 [winenara_spiders](./winenara_scrapy/tutorial/spiders/winenara_spider.py) 파일을 수정한다.

### 상품목록페이지에서 각 상품 상세페이지 링크 추출
<img src="./assets/winenara_mainpage.png" width="500" height="500">

위에서 각 상세페이지의 주소를 가져오고, 모든 상세페이지에서 정보를 가져왔다면 다음 페이지 링크를 이용해 다음 페이지로 넘어간다.

### 상세페이지에서 와인 정보 추출
<img src="./assets/winenara.png" width="1000" height="500">

위의 이미지는 와인나라의 상세페이지이다. 상세페이지로부터 아래의 값들을 가져온다.
- url: 해당 페이지의 url
- price: 가격
- name: 한글 이름
- en_name: 영어 이름
- img_url: 이미지 url 리스트
- features: 와인에 대한 정보 딕셔너리(바디, 산도, 타닌, 알코올 등)
- tag: 와인에 대한 정보 리스트(와인의 종류, 원산지, 포도품종)
- rating: vivino 점수(1~5점)
- vivino_link: vivino url (vivino는 전세계의 와인 데이터베이스 웹페이지이다.)

예를 들어 위의 이미지에 대해서는 아래와 같이 만들어진다.
```json
{"url": "https://www.winenara.com/shop/product/product_view?product_cd=29E033",
 "price": "50,000원", 
 "name": "SET)페데럴리스트 카베르네 소비뇽 원통 패키지", 
 "en_name": "SET)THE FEDERALIST CABERNET SAUVIGNON", 
 "img_url": ["https://www.winenara.com/uploads/product/550/e0705cecad48a9e01710b6fbf7b2f8c5.png"], 
 "features": {"바디": "중간"},
 "feature_img_url": ["https://www.winenara.com/uploads/editor/202212081007511218373150.jpg"],
 "tag": ["레드", "미국", "카베르네 소비뇽"], 
 "rating": "3.8", 
 "vivino_link": "https://www.vivino.com/federalist-cabernet-sauvignon/w/3048981", "images": [{"url": "https://www.winenara.com/uploads/product/550/e0705cecad48a9e01710b6fbf7b2f8c5.png", "path": "product\\550\\e0705cecad48a9e01710b6fbf7b2f8c5.png", "checksum": "e04ed429fd54db2478136c3e03ab592f", "status": "downloaded"}]}
```

----------------- 코드 실행 ------------------

In [1]:
%cd winenara_scrapy

c:\Users\chois\Desktop\chatwine\winenara_scrapy


In [2]:
%cd tutorial

c:\Users\chois\Desktop\chatwine\winenara_scrapy\tutorial


크롤링 과정에서 img_url의 이미지가 로컬에 저장된다. 이를 위해 [settings.py](./winenara_scrapy/tutorial/settings.py)에 이미지가 저장될 경로를 설정한다. settings.py에 경로를 변경해서 아래 코드를 추가해준다.
```python
ITEM_PIPELINES = {'tutorial.pipelines.CustomImagesPipeline': 1}
IMAGES_STORE = 'C:/Users/chois/Desktop/Audrey/chatwine/winenara_scrapy/assets/img'
```

또한 이미지 저장에 있어서 이미지의 이름을 변경해주기 위해 [pipelines.py](./winenara_scrapy/tutorial/pipelines.py)에 CustomImagePipeline 클래스를 추가해준다. 코드는 아래와 같다.
```python
import os
from urllib.parse import urlparse

import scrapy
from scrapy.pipelines.images import ImagesPipeline

class CustomImagesPipeline(ImagesPipeline):
    def file_path(self, request, response=None, info=None, *, item=None):
        path = urlparse(request.url).path
        path_parts = path.split('/')
        filename = os.path.join(*path_parts[-3:])
        return filename

    def get_media_requests(self, item, info):
        if 'img_url' in item:
            for image_url in item['img_url']:
                yield scrapy.Request(image_url)
```

아래 코드를 실행하여 얻은 결과는 [winenara.json](./winenara_scrapy/winenara.json)에서 확인할 수 있다.

In [3]:
!scrapy crawl winenara -o new_winenara.json # winenara spider를 실행하여 결과를 winenara.json에 저장, 터미널에서 실행해야해요.

2023-07-15 03:00:13 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: tutorial)
2023-07-15 03:00:13 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.12, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.1, Platform Windows-10-10.0.22621-SP0
Usage
=====
  scrapy crawl [options] <spider>
crawl: error: running 'scrapy crawl' with more than one spider is not supported


아래는 파일 feature_img_url을 [저장 경로](./winenara_scrapy/assets/img/feature_img/)에 저장하는 코드이다.(위의 CustomImagePipeline을 수정하면 된다. 그런데 깜빡했다!)

In [30]:
%cd winenara_scrapy

c:\Users\chois\Desktop\chatwine\winenara_scrapy


In [31]:
%cd tutorial

c:\Users\chois\Desktop\chatwine\winenara_scrapy\tutorial


아래는 와인 이미지를 ./winenara_scrapy/assets/img/product에 저장하는 코드이다.

In [11]:
import pandas as pd

# Load the two json files into pandas dataframes
new_df = pd.read_json('../new_winenara.json')
old_df = pd.read_json('../winenara.json')

# Ensure 'name' column exists in both dataframes
if 'name' not in new_df.columns or 'name' not in old_df.columns:
    print("Both dataframes must contain 'name' column.")
    exit(1)

# Extract the 'name' column from each dataframe and convert to set for comparison
names_df1 = set(new_df['name'])
names_df2 = set(old_df['name'])

# Identify the names that have been added and removed
removed_names = names_df2 - names_df1
added_names = names_df1 - names_df2

# Print the results
print("Added names:", added_names)
print("Removed names:", removed_names)


Added names: {'포추네이트 선 디플로맷', '에라주리즈 맥스 소비뇽블랑', '맥스 에이트 (8, VIII)', '포추네이트 선 워리어', '포추네이트 선 드리머', '디아블로 소비뇽 블랑'}
Removed names: {'SET)빌라엠비앙코 + 글라스2개 윈터패키지'}


In [8]:
# Merge the two dataframes on the 'name' column
merged_df = pd.merge(new_df, old_df, on='name', suffixes=('_new', '_old'))

# Iterate over the rows and check for price changes
for index, row in merged_df.iterrows():
    if row['price_old'] != row['price_new']:
        print(f"The price of '{row['name']}' has changed from {row['price_old']} to {row['price_new']}.")

The price of '디아블로 로제' has changed from 12,900원 to 10,900원.
The price of '샤또 루빈 라이온 앤 드래곤 프로방스 로제' has changed from 45,000원 to 49,000원.


In [4]:
import json
import os
import requests
from tqdm import tqdm

# JSON 파일 열기
with open('../new_winenara.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 저장 경로 지정
save_path = '../assets/img/product'

# 모든 아이템에 대해 반복
for item in tqdm(data):
    image_url = item['img_url'][0]  # feature_img_url에서 이미지 URL 가져오기
    image_name = item['name'] + '.png'  # 이미지 이름 지정하기

    # URL에서 이미지 다운로드하기
    try:
        response = requests.get(image_url)

        # 이미지를 파일로 저장하기
        with open(os.path.join(save_path, image_name), 'wb') as img_file:
            img_file.write(response.content)
    except:
        print('Cannot download image from:', item['name'])
        continue   

Cannot download image from: 부온크리스티아니 "더 코어" 나파 밸리 카베르네 소비뇽


아래는 와인 feature 이미지를 ./winenara_scrapy/assets/img/feature_img에 저장하는 코드이다.

In [ ]:
import json
import os
import requests
from tqdm import tqdm

# JSON 파일 열기
with open('./new_winenara.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 저장 경로 지정
save_path = '../assets/img/feature_img'

# 모든 아이템에 대해 반복
for item in tqdm(data):
    image_url = item['feature_img_url'][0]  # feature_img_url에서 이미지 URL 가져오기
    image_name = item['name'] + '.png'  # 이미지 이름 지정하기

    # URL에서 이미지 다운로드하기
    try:
        response = requests.get(image_url)

        # 이미지를 파일로 저장하기
        with open(os.path.join(save_path, image_name), 'wb') as img_file:
            img_file.write(response.content)
    except:
        print('Cannot download image from:', item['name'])
        continue   


아래는 저장한 feature_img로 부터 OCR을 통해 정보를 가져오는 단계이다. 
feature 이미지는 아래와 같다.

<img src="./assets/winenara_feature_img.png" width="500" height="1000">

가져올 수 있는 feature에는 아래와 같은 값이 있다.
- 대표 맛과 향
- 핸들링
- 와이너리이야기
- 페어링

In [5]:
import os
import configparser

In [112]:
%cd ../

c:\Users\chois\Desktop\chatwine


In [113]:
%pwd # 'c:\\Users\\chois\\Desktop\\chatwine'

'c:\\Users\\chois\\Desktop\\chatwine'

In [6]:
config = configparser.ConfigParser()
config.read('./secrets.ini')

['./secrets.ini']

In [7]:
NAVER_OCR_API_URL = config['NAVER_OCR']['API_URL']
NAVER_OCR_SECRET_KEY = config['NAVER_OCR']['SECRET_KEY']

In [8]:
import requests
import uuid
import time
import json
import os
from tqdm import tqdm
import re

class NaverOCR:
    def __init__(self, save_root_path):
        self.api_url = NAVER_OCR_API_URL
        self.secret_key = NAVER_OCR_SECRET_KEY
        self.save_root_path = save_root_path
        os.makedirs(save_root_path, exist_ok=True)

        request_json = {
        'images': [
        {
        'format': 'jpg',
        'name': 'demo'
        }
        ],
        'requestId': str(uuid.uuid4()),
        'version': 'V2',
        'timestamp': int(round(time.time() * 1000))
        }

        self.payload = {'message': json.dumps(request_json).encode('UTF-8')}
        
        self.headers = {
        'X-OCR-SECRET': self.secret_key
        }

    def get_ocr(self, img_file):
        files = [
        ('file', open(img_file,'rb'))
        ]
        filename = os.path.basename(img_file)
        response = requests.request("POST", self.api_url, headers=self.headers, data = self.payload, files = files)
        with open(os.path.join(self.save_root_path, filename.replace('.png', '.json')), 'w', encoding='utf-8') as f:
            f.write(response.text)
        return response.text

In [46]:
ocr = NaverOCR()

아래 코드를 통해서 feature_img를 Naver OCR에 통과시키고, 그 response를 ./winenara_scrapy/assets/raw_ocr_results/ 에 저장한다.

In [55]:
for img_file_path in tqdm(img_file_path_list):
    ocr.get_ocr(os.path.join(img_file_root_path, img_file_path))

  0%|          | 0/472 [00:00<?, ?it/s]

100%|██████████| 472/472 [14:43<00:00,  1.87s/it]


아래코드는 Naver OCR을 통해 저장한 raw response를 실제 우리가 원하는 feature로 정제하여 json으로 저장하는 코드이다. 정제된 feature는 ./winenara_scrapy/assets/ocr_results/ 에 저장된다. feature는 위에서 말한것 처럼 다음과 같다. 

- 대표 맛과 향
- 핸들링
- 와이너리이야기
- 페어링

In [102]:
def split_text_to_dict(text, keywords):
    keyword_positions = []
    
    for keyword in keywords:
        keyword_index = text.find(keyword)
        if keyword_index != -1:
            keyword_positions.append((keyword, keyword_index))

    keyword_positions.sort(key=lambda x: x[1])
    
    split_dict = {}
    current_index = 0
    for keyword, position in keyword_positions:
        if current_index != 0:
            last_keyword, _ = keyword_positions[keyword_positions.index((keyword, position))-1]
            split_dict[last_keyword] = text[current_index:position].strip()
        current_index = position + len(keyword)

    if current_index != 0:
        last_keyword, _ = keyword_positions[-1]
        split_dict[last_keyword] = text[current_index:].strip()

    if "대표 맛과 향" in split_dict:
        # Remove English letters
        split_dict["대표 맛과 향"] = re.sub(r'[a-zA-Z]', '', split_dict["대표 맛과 향"])

    if "핸들링" in split_dict:
        # Split by space, reorder, and join again
        split_dict["핸들링"] = split_dict["핸들링"].strip()
        
        special_chars = '()YG오'
        for special_char in special_chars:
            split_dict["핸들링"] = split_dict["핸들링"].replace(special_char, '')
        handling_parts = split_dict["핸들링"].split(' ')
        handling_parts = [string for string in handling_parts if string != '']

        if len(handling_parts) == 11:
            order = [0, 5, 6, 1, 7, 2, 8, 3, 9, 4, 10]
        elif len(handling_parts) == 10:
            order = [0, 5, 1, 6, 2, 7, 3, 8, 4, 9]
        elif len(handling_parts) == 9:
            order = [0, 4, 5, 1, 6, 2, 7, 3, 8]
        elif len(handling_parts) == 8:
            if '아로마' in handling_parts:
                order = [0, 3, 4, 1, 5, 6, 2, 7]
            else:
                order = [0, 4, 1, 5, 2, 6, 3, 7]
        elif len(handling_parts) == 7:
            order = [0, 3, 4, 1, 5, 2, 6]
        elif len(handling_parts) == 6:
            order = [0, 3, 1, 4, 2, 5]
        else:
            print(len(handling_parts), handling_parts)
            print(split_dict["핸들링"])
        reordered_handling_parts = [handling_parts[i] for i in order]
        
        split_dict["핸들링"] = ' '.join(reordered_handling_parts)

    return split_dict

  0%|          | 0/472 [00:00<?, ?it/s]

100%|██████████| 472/472 [00:01<00:00, 340.14it/s]


In [ ]:
keywords = ['대표 맛과 향', '핸들링', '페어링', '와이너리이야기']
raw_ocr_file_root_path = '../assets/raw_ocr_results'
ocr_file_root_path = '../assets/ocr_results'
os.makedirs(ocr_file_root_path, exist_ok=True)
raw_ocr_file_path_list = os.listdir(raw_ocr_file_root_path)

for raw_ocr_file_path in tqdm(raw_ocr_file_path_list):
    filename = os.path.basename(raw_ocr_file_path)
    with open(os.path.join(raw_ocr_file_root_path, raw_ocr_file_path), 'r', encoding='utf-8') as f:
        dictionary = json.load(f)
        unified_text = ''
        for text in dictionary['images'][0]['fields']:
            unified_text += text['inferText'] + ' '
        split_dict = split_text_to_dict(unified_text, keywords)

    with open(os.path.join(ocr_file_root_path, filename), 'w', encoding='utf-8') as f:
        json.dump(split_dict, f, ensure_ascii=False, indent=4)

이렇게 수집한 OCR 데이터는 완전한 데이터가 아닙니다. 그림에서 글자를 추출하거나 잘못 인식된 글자들이 있습니다. 이들의 수정은 사람이 직접해야합니다.

### 와인나라 와인의 vivino를 통한 특징 추출

와인나라에서는 판매하는 와인에 대한 feature(바디감, 당도 등...)를 제공하고 있다. 하지만 작성이 안되어있는 와인들도 많다. 따라서 이 정보들을 가져오기 위해서 아래 두 가지의 외부 데이터를 사용할 것이다. 
1. [vivino api](https://github.com/aptash/vivino-api)
2. [X-wine dataset](https://github.com/rogerioxavier/X-Wines)

이를 위해 먼저 vivino api github을 클론한다.

In [ ]:
%cd .. # pwd가 chatwine이 되도록 함
!git clone "https://github.com/aptash/vivino-api.git"

이후 [vivino.js](./vivino-api/vivino.js)의 244 line을 일부 수정한다. 이는 각 와인 검색의 json파일이 저장될 경로를 지정하기 위함이다.
```javascript
const outFile = fs.createWriteStream('./output/'+name+'.json');
```

vivino api는 아래와 같이 사용할 수 있다.
```shell
cd vivino-api
node vivino.js --name=ad-vivum-cabernet-sauvignon --minPrice=10 --maxPrice=25
```

이것에 대한 결과는 [./output/ad-vivum-cabernet-sauvignon.json](vivino-api/output/ad-vivum-cabernet-sauvignon.json)과 같이 json파일로 얻을 수 있다.
```json
{
  "vinos": [
    {
      "name": "AD VIVUM Cabernet Sauvignon",
      "link": "https://www.vivino.com/US-CA/en/wines/2493776",
      "thumb": "https://images.vivino.com/thumbs/O_7LrDsLQSOxl2CqVfRHVA_pb_300x300.png",
      "country": "United States",
      "region": "Napa Valley",
      "average_rating": 4.4,
      "ratings": 261,
      "price": 186.16
    }
  ],
  "status": "FULL_DATA"
}
```

이제 [winenara.json](./winenara_scrapy/winenara.json)의 와인들에 대해서 vivino api를 통해 검색하는 코드를 보자.

검색을 위해서는 winenara.json의 vivino_link를 활용한다. url의 쿼리에서 와인 이름을 추출하여 이를 이용하여 검색한다. 예시는 아래와 같다.

> vivino_link: https://www.vivino.com/monopole-la-rioja-blanco-seco-clasico/w/1925448 <br>
> -> 추출한 와인이름: monopole-la-rioja-blanco-seco-clasico

In [2]:
import json
from tqdm import tqdm

# Read the winenara json file
with open('./winenara_scrapy/winenara.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [3]:
%cd vivino-api

c:\Users\chois\Desktop\chatwine\vivino-api


아래 코드처럼 멀티프로세싱을 활용하여 와인 데이터를 검색한다. 그 결과는 "./vivino-api/output"에 저장된다.

In [ ]:
import concurrent.futures
from tqdm import tqdm

import os
file_list = os.listdir('./output')
json_list = [file.replace('.json', '') for file in file_list]

data_list   = []
for wine_data  in data:
    if wine_data['vivino_link'].split('/')[-3] not in json_list:
        data_list.append(wine_data)

def fetch_wine_data(wine_data):
    wine_name = wine_data['vivino_link'].split('/')[-3]
    !node vivino.js "--name={wine_name}"

# Use a ThreadPoolExecutor to run fetch_wine_data in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    list(tqdm(executor.map(fetch_wine_data, data_list), total=len(data_list)))

### 와인나라 와인의 데이터 통합

아래 과정은 naive하게 크롤링한 와인나라 와인의 통합된 형태로 변경하고, json을 저장하는 과정입니다.

In [13]:
%cd ../../

c:\Users\chois\Desktop\chatwine


In [22]:
import os
import json
import pandas as pd

In [23]:
with open('./winenara_scrapy/winenara.json', 'r', encoding='utf-8') as f:
    wine_list = json.load(f)
df = pd.DataFrame(wine_list, columns=['price', 'name', 'en_name','rating', 'url', 'tag', 'features', 'img_url', 'vivino_link'])

In [24]:
wine_type_list = ['레드', '로제', '스파클링', '화이트', '디저트', '주정강화']
country_list = ['기타 신대륙', '기타구대륙', '뉴질랜드', '독일', '미국', '스페인', '아르헨티나', '이탈리아', '칠레', '포루투칼', '프랑스', '호주']

In [25]:
drop_idx = []
for idx in range(len(df)):
    if (df['tag'][idx][0] not in wine_type_list) or (df['tag'][idx][1] not in country_list):
        drop_idx.append(idx)
df = df.drop(drop_idx)
df = df.reset_index(drop=True)


In [26]:
df['wine_type'] = ''
df['country'] = ''
df['sweetness'] = ''
df['body'] = ''
df['alcohol'] = ''
for idx in range(len(df)):
    df['wine_type'][idx] = df['tag'][idx][0]
    df['country'][idx] = df['tag'][idx][1]
    for key, value in df['features'][idx].items():
        if key == '당도':
            df['sweetness'][idx] = value
        elif key == '알코올':
            df['alcohol'][idx] = value
        elif key == '바디':
            df['body'][idx] = value
        else:
            raise ValueError('Wrong feature key')

In [27]:
df = df.drop(columns=['tag', 'features'])

In [28]:
df['price'] = df['price'].replace('원', '', regex=True)
df['price'] = df['price'].replace(',', '', regex=True)

In [29]:
sweetness_dict = {'':-1, '드라이':1, '미디움드라이':2, '미디엄':3, '미디움스윗':4, '스윗':5}
body_dict = {'':-1, '가벼움':1, '약간가벼움':2, '중간':3, '약간무거움':4, '무거움':5}
alcohol_dict = {'':-1, '중간(12~13%)':3,'높음(14%+)':5}


In [30]:
for idx in range(len(df)):
    df['sweetness'][idx] = sweetness_dict[df['sweetness'][idx]]
    df['body'][idx] = body_dict[df['body'][idx]]
    df['alcohol'][idx] = alcohol_dict[df['alcohol'][idx]]

In [23]:
df.columns

Index(['price', 'name', 'en_name', 'rating', 'url', 'img_url', 'vivino_link',
       'wine_type', 'country', 'sweetness', 'body', 'alcohol'],
      dtype='object')

In [10]:
df.head()

,price,name,en_name,rating,url,img_url,vivino_link,wine_type,country,sweetness,body,alcohol
0,38800,샹동 로제 브룻,Chandon Rose brut,3.7,https://www.winenara.com/shop/product/product_...,[https://www.winenara.com/uploads/product/550/...,https://www.vivino.com/US-CA/en/chandon-califo...,로제,호주,-1,-1,-1
1,200000,티터 토터 카베르네소비뇽,TEETER-TOTTER CABERNET SAUVIGNON,4.3,https://www.winenara.com/shop/product/product_...,[https://www.winenara.com/uploads/product/550/...,https://www.vivino.com/US-CA/en/teeter-totter-...,레드,미국,-1,4,-1
2,1100000,아르망 드 브리냑 브룻 골드,Armand De Brignac Brut Gold,4.4,https://www.winenara.com/shop/product/product_...,[https://www.winenara.com/uploads/product/550/...,https://www.vivino.com/US-CA/en/armand-de-brig...,스파클링,프랑스,2,-1,-1
3,595000,크룩 로제,Krug brut Rose,4.6,https://www.winenara.com/shop/product/product_...,[https://www.winenara.com/uploads/product/550/...,https://www.vivino.com/US-CA/en/krug-brut-rose...,로제,프랑스,-1,-1,-1
4,100000,도멘 퐁사르 슈발리에 마랑지 1er 라 푸시에,DOMAINE PONSARD CHEVALIER MARANGES 1ER LA FUSS...,4.2,https://www.winenara.com/shop/product/product_...,[https://www.winenara.com/uploads/product/550/...,https://www.vivino.com/US-CA/en/ponsard-cheval...,레드,프랑스,-1,3,-1


In [31]:
winenara_attribute = ['price', 'name', 'en_name', 'rating', 'url', 'wine_type', 'country', 'sweetness', 'body', 'alcohol', 'img_url', 'vivino_link']
unified_attribute = ['url', 'site_name', 'price', 'name', 'en_name', 'img_url', 'body', 'acidity', 'tannin', "sweetness", "alcohol", "wine_type", 'country', 'grape', 'rating', 'pickup_location', 'vivino_link', 'flavor_description', 'pairing']
unified_dict = {'url':'url', 'site_name':'', 'price':'price', 'name':'name', 'en_name':'en_name', 'img_url':'img_url', 'body':'body', 'acidity':'', 'tannin':'', "sweetness":'sweetness', "alcohol":'alcohol', "wine_type":'wine_type', 'country':'country', 'grape':'', 'rating':'rating', 'pickup_location':'', 'vivino_link':'vivino_link', 'flavor_description':'', 'pairing':''}
data_list = []
for idx in range(len(df)):
    single_data = {}
    for unified_attr, winenara_attr in unified_dict.items():
        if winenara_attr != '':
            if type(df[winenara_attr][idx]) == list:
                single_data[unified_attr] = ', '.join(df[winenara_attr][idx])
            else:
                single_data[unified_attr] = df[winenara_attr][idx]            
        elif unified_attr == 'site_name':
            single_data[unified_attr] = 'winenara'
        else:
            single_data[unified_attr] = ''
    data_list.append(single_data)

In [12]:
data_list[0]

{'url': 'https://www.winenara.com/shop/product/product_view?product_cd=02A287',
 'site_name': 'winenara',
 'price': '38800',
 'name': '샹동 로제 브룻',
 'en_name': 'Chandon Rose brut',
 'img_url': 'https://www.winenara.com/uploads/product/550/8565729ded4f5e4df31fb15707c8a971.png',
 'body': -1,
 'acidity': '',
 'tannin': '',
 'sweetness': -1,
 'alcohol': -1,
 'wine_type': '로제',
 'country': '호주',
 'grape': '',
 'rating': '3.7',
 'pickup_location': '',
 'vivino_link': 'https://www.vivino.com/US-CA/en/chandon-california-rose-california/w/1385746',
 'flavor_description': '',
 'paring': ''}

아래는 feature_img로 부터 뽑은 feature(./winenara_scrapy/assets/ocr_results)를 data_list에 추가하는 코드입니다.

In [34]:
ocr_file_root_path = './winenara_scrapy/assets/ocr_results/'
ocr_file_list = os.listdir(ocr_file_root_path)

for data in data_list:
    data['flavor_description'] = ''
    data['pairing'] = ''
    if data['name']+'.json' in ocr_file_list:
        with open(os.path.join(ocr_file_root_path, data['name']+'.json'), encoding='utf-8') as f:
            ocr_result = json.load(f)
        data['flavor_description'] = ocr_result['대표 맛과 향'] if '대표 맛과 향' in ocr_result.keys() else ''
        data['pairing'] = ocr_result['페어링'] if '페어링' in ocr_result.keys() else ''

In [35]:
with open('./data/winenara.json', 'w', encoding='utf-8') as json_file:
    for data in data_list:
        json_file.write(json.dumps(data, ensure_ascii=False) + '\n')

In [36]:
with open('./data/unified_wine_data.json', 'w', encoding='utf-8') as json_file:
    for data in data_list:
        json_file.write(json.dumps(data, ensure_ascii=False) + '\n')

## 와인앤모어 크롤링

와인앤모어 크롤링을 위해서는 [Selenium](https://selenium-python.readthedocs.io/)을 사용한다.

Scrapy를 사용해서도 할 수 있다.(아마도?) 근데 나는 scrapy로 로그인과 성인인증을 뚫지 못해서 사람이 브라우저와 interaction하는 것과 비슷한 Selenium을 사용하였다.

In [1]:
import os
from time import sleep
import configparser

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

아래 코드를 실행하면 크롬이 열리고, 와인앤모어 로그인 페이지로 접속하게 된다.

In [2]:
# Setup the driver. This one uses chrome with some options and a path to the chromedriver
driver = webdriver.Chrome()

# implicitly_wait tells the driver to wait before throwing an exception
driver.implicitly_wait(30)

# go to the login page
driver.get('https://www.wineandmore.co.kr/member/login.php') # 와인앤모어 로그인페이지 접속

로컬파일로부터 와인앤모어 아이디, 비밀번호 가져오기

In [3]:
config = configparser.ConfigParser()
config.read('./secrets.ini')

['./secrets.ini']

In [4]:
wineandmore_id = config['WINE_AND_MORE']['ID']
wineandmore_password = config['WINE_AND_MORE']['PASSWORD']

아래 코드를 실행하면 아이디, 패스워드를 입력하는 부분이 채워진다.

In [7]:
# the driver.get method will navigate to a page given by the URL
username = driver.find_element(By.ID, "loginId")
password = driver.find_element(By.ID, "loginPwd")

# Target the form elements and send_keys to simulate key strokes
username.send_keys(wineandmore_id)
password.send_keys(wineandmore_password)

# Submit form
driver.find_element(By.CLASS_NAME, 'member_login_order_btn').click()

# Allow time for the website to load
sleep(5)

### 상품목록페이지에서 각 상품 상세페이지 링크 추출

상품리스트 페이지는 다음과 같은 url구조를 갖고있다. idx에 페이지 번호가 들어가게된다.

https://www.wineandmore.co.kr/goods/goods_list.php?page={idx}&cateCd=001

페이지는 1~28까지 있으므로 순회하며 크롤링한다.(정말 나중에는 바꿔줘야하는 안좋은 코드다..)

<img src="./assets/wineandmore_mainpage.png" width="500" height="500">

### 상품 상세페이지에서 와인 데이터 가져오기
<img src="./assets/wineanndmore_detailpage.png" width="800" height="500">

위의 이미지는 와인앤모어의 상세페이지이다. 상세페이지로부터 아래의 값들을 가져온다.
- url: 해당 페이지의 url
- name: 한글 이름
- price: 가격
- feature_img_url: 와인에 대한 상세 정보가 담겨있는 이미지 url
- img_url: 이미지 url 리스트
- pickup_info: 해당 와인의 픽업이나 구매가 가능한 와인앤모어 오프라인 매장

예를 들어 위의 이미지에 대해서는 아래와 같이 만들어진다.
```json
"url": "https://www.wineandmore.co.kr/goods/goods_view.php?goodsNo=1000002401", 
"name": "장 미셸 기불로 사비니 레 본 블랑", 
"price": "53,000", 
"feature_img_url": "https://shinsegaelnb.cdn-nhncommerce.com/data/editor/goods/230426/f2dcc9b3248c7d163cbb29c4d452cb4b_173014.jpg",
"img_url": "https://shinsegaelnb.cdn-nhncommerce.com/data/goods/23/02/08/1000002401/1000002401_detail_048.jpg", 
"pickup_info": ["와인앤모어 교대역점", "와인앤모어 다산점", "와인앤모어 서울대입구역점", "와인앤모어 스타필드 위례점", "와인앤모어 여의도점"]
```

여기서 feature_img_url이라는 것이 있는데, 이는 아래 사진과 같다. 나중에 OCR을 통해 데이터를 뽑아와야한다.

<img src="./assets/wineandmore_feature_img.png" width="500" height="500">

아래 함수는 상세페이지에서 데이터를 가져오는 코드이다.

In [8]:
def get_detail_info(detail_url):
    driver.get(detail_url)
    # Allow time for the website to load
    sleep(10)
    data_dict = {}
    data_dict['url'] = driver.current_url

    name = driver.find_element(By.CSS_SELECTOR, ".item_detail_tit h3")
    data_dict['name'] = name.text if name else ""

    price = driver.find_element(By.CSS_SELECTOR, "dl.item_price strong strong")
    data_dict['price'] = price.text if price else ""

    data_dict['feature_img_url'] = driver.find_elements(By.CSS_SELECTOR, '.js-smart-img')[3].get_attribute('src')

    img_url = driver.find_element(By.CSS_SELECTOR, ".zoom_layer_open img.middle")
    data_dict['img_url'] = img_url.get_attribute('src')

    pickup_info_list = driver.find_elements(By.CSS_SELECTOR, ".form_element select option")
    data_dict['pickup_info'] = [pickup_info.text for pickup_info in pickup_info_list if pickup_info.text != "매장선택"]

    return data_dict

아래 코드를 실행하면 페이지를 순회하면 크롤링을 진행하고, 이를 data_list에 저장한다.

In [10]:
# Navigate to product list page
data_list = []
for idx in range(1, 30):
    driver.get(f'https://www.wineandmore.co.kr/goods/goods_list.php?page={idx}&cateCd=001')
    sleep(10)
    product_links = driver.find_elements(By.CSS_SELECTOR, "div.item_photo_box a")
    links = set()
    for link in product_links:
        links.add(link.get_attribute('href'))
    for detail_url in tqdm(links):
        data_list.append(get_detail_info(detail_url))

100%|██████████| 20/20 [03:43<00:00, 11.17s/it]


아래 코드를 통해 수집한 데이터를 [json 파일](./data/wine_and_more.json)로 저장한다.

In [11]:
import json

with open('./data/wine_and_more/wine_and_more_7new.json', 'w', encoding='utf-8') as json_file:
    for data in data_list:
        json_file.write(json.dumps(data, ensure_ascii=False) + '\n')

In [14]:
import pandas as pd

# Load the two json files into pandas dataframes
new_df = pd.read_json('./data/wine_and_more/wine_and_more_7new.json', lines=True)
old_df = pd.read_json('./data/wine_and_more/wine_and_more.json', lines=True)

# Ensure 'name' column exists in both dataframes
if 'name' not in new_df.columns or 'name' not in old_df.columns:
    print("Both dataframes must contain 'name' column.")
    exit(1)

# Extract the 'name' column from each dataframe and convert to set for comparison
names_df1 = set(new_df['name'])
names_df2 = set(old_df['name'])

# Identify the names that have been added and removed
removed_names = names_df2 - names_df1
added_names = names_df1 - names_df2

# Print the results
print("Added names:", added_names)
print("Removed names:", removed_names)


Added names: {'[W&M ONLY] 티나찌 아마로네 델라 발폴리첼라 DOCG', 'LA 다저스 까베르네 소비뇽', '와비 사비 스페이스 버블스 화이트', '[7월행사] 루이자도 부르고뉴 알리고떼', '치아보베르통 바롤로 라모라', '스칼리올라 지오코 피오레', '뉴욕 멧츠 까베르네 소비뇽', '[단독행사] 쌩 클레어 파이오니어 블록 소비뇽 블랑', '샤또 라랑드 2015', '[단독행사] 스칼리올라 프리모 피오레 모스카토 다스티', '[GCC 패키지] 샤또 깔롱 세귀르 5입 세트 (89+95+03+10+16)', '앙드레 끌루에 상파뉴 브뤼 로제', '톨라이니 레짓 IGT', '샤또 마고 2011', '캔자스 시티 로열스 까베르네 소비뇽', '[7월행사] 블랭크 캔버스 앱스트랙트 소비뇽 블랑', '까르페스 데 에고메이 크리안자', '장 미셸 기불로 사비니 1er 크뤼 푸르노', '칸티나 꼴리 에오가네이 모스카토 스푸만테 돌체', '코노수르 비씨클레타 까르미네르', '[단독행사] JP 슈네 딜리셔스 로제', '알레산드로 리베또 바롤로', '로스피탈레 드 가쟁', '까테나 자파타 안젤리카 자파타 샤도네이', '[7월행사] 니콜라스 푸이야트 셀렉션 브뤼', '플라네타 샤도네이', '세인트 루이스 카디널스 까베르네 소비뇽', '투핸즈 아레스', '코노수르 비씨클레타 샤도네이', '디트로이트 타이거즈 까베르네 소비뇽', '파스쿠아 프리잔테 모스카토 다스티', '[W&M ONLY] 빌라도리아 바롤로 DOCG', '샤또 몽페라 화이트', '상파뉴 루이 뒤몽 로제', '아타랑이 크림슨 피노누아', '뉴욕 양키즈 까베르네 소비뇽', '[단독행사] 쌩 클레어 비카스 초이스 소비뇽 블랑', '[W&M ONLY] 빌라도리아 바르바레스코 DOCG', '칸티 모스카토 다스티 DOCG 골드 에디션', '샤또 글로리아 2017', '섹스탕 푸푸네트', '와비 사비 사쿠라 로제 펫 낫', '[7월행사] 도미니오 우노 마스 까바 브뤼', '샤또 라퐁 로쉐 2017', '689 화이

In [15]:
# Merge the two dataframes on the 'name' column
merged_df = pd.merge(new_df, old_df, on='name', suffixes=('_new', '_old'))

# Iterate over the rows and check for price changes
for index, row in merged_df.iterrows():
    if row['price_old'] != row['price_new']:
        print(f"The price of '{row['name']}' has changed from {row['price_old']} to {row['price_new']}.")

The price of '우드브릿지 샤도네이' has changed from 25,000 to 19,800.


In [12]:
driver.close()

와인앤모어 feature_img를 다운로드받아 ./data/wine_and_more/img에 저장하자.

In [34]:
%cd ../../

c:\Users\chois\Desktop\chatwine


In [45]:
import json
import os
import requests

# JSON 파일 열기
data_list = []
with open('./data/wine_and_more/wine_and_more_new.json', 'r', encoding='utf-8') as f:
    while True:
        line = f.readline()
        if not line:
            break
        data_list.append(json.loads(line))

# 저장 경로 지정
save_path = './data/img/wine_and_more'
os.makedirs(save_path, exist_ok=True)

# 모든 아이템에 대해 반복
for item in data_list:
    image_url = item['feature_img_url']  # feature_img_url에서 이미지 URL 가져오기
    image_name = item['name'] + '.png'  # 이미지 이름 지정하기

    # URL에서 이미지 다운로드하기
    try:
        response = requests.get(image_url)

        # 이미지를 파일로 저장하기
        with open(os.path.join(save_path, image_name), 'wb') as img_file:
            img_file.write(response.content)
    except:
        print('Cannot download image from:', item['name'])
        continue   

Cannot download image from: [클리어런스/온라인단독] [GCC 패키지] 샤또 깔롱 세귀르 5입 세트 (89+95+03+10+16)
Cannot download image from: [클리어런스/온라인단독] [GCC 패키지] 샤또 레오빌 라 꺄스 00 + 샤또 피작 10 + 샤또 팔머 05 750ml 세트
Cannot download image from: [클리어런스/온라인단독] [GCC 패키지] 샤또 뽕떼 까네 6입 세트 (90+94+95+96+98+99)
Cannot download image from: [클리어런스/온라인단독] [GCC 패키지] 샤또 오존 750ml 3입세트 (09+10+15)
Cannot download image from: [클리어런스/온라인단독] [GCC 패키지] 샤또 레오빌 라 꺄스 00 + 샤또 피작 10 + 샤또 팔머 05 1.5L 세트


In [5]:
# JSON 파일 열기
data_list = []
with open('./data/wine_and_more.json', 'r', encoding='utf-8') as f:


In [20]:
# convert DataFrame to list of dicts
data_list = df.to_dict(orient='records')

# write to file
with open('./data/wine_and_more_new.json', 'w', encoding='utf-8') as json_file:
    for data in data_list:
        json_file.write(json.dumps(data, ensure_ascii=False) + '\n')

In [8]:
import pandas as pd

In [12]:
df = pd.read_json("./data/wine_and_more_new.json", lines=True)

In [14]:
def replace_wine(query):
    return query.replace('./data/wine_and_more/img/', 'https://github.com/audreyaiai/chatwine-dev/blob/main/data/wine_and_more/img/').replace('.png', '.png?raw=True')

In [16]:
df['img_url'] = df['img_url'].apply(replace_wine)

In [18]:
df['img_url'][0]

'https://github.com/audreyaiai/chatwine-dev/blob/main/data/wine_and_more/img/투핸즈_아레스_매그넘.png?raw=True'

이제 다운로드한 feature image에 대해 ocr을 진행한다.

In [ ]:
ocr = NaverOCR(save_root_path='./data/wine_and_more/raw_ocr_results')

아래 코드를 통해서 feature_img를 Naver OCR에 통과시키고, 그 response를 .data/wine_and_more/raw_ocr_results/ 에 저장한다.

In [52]:
img_file_root_path = './data/wine_and_more/img'
img_file_path_list = os.listdir(img_file_root_path)
for img_file_path in tqdm(img_file_path_list):
    ocr.get_ocr(os.path.join(img_file_root_path, img_file_path))

  0%|          | 0/551 [00:00<?, ?it/s]

100%|██████████| 551/551 [15:25<00:00,  1.68s/it]


아래코드는 Naver OCR을 통해 저장한 raw response를 실제 우리가 원하는 feature로 정제하여 json으로 저장하는 코드이다. 
정제된 feature는 ./data/wine_and_more/ocr_results/ 에 저장된다. feature는 아래와 같다.

- name
- en_name
- flavor description
- product info
- food pairings

In [79]:
def split_text_to_dict(text, keywords):
    """Split text to dictionary by keywords"""
    keyword_positions = []
    
    for keyword in keywords:
        keyword_index = text.find(keyword)
        if keyword_index != -1:
            keyword_positions.append((keyword, keyword_index))

    keyword_positions.sort(key=lambda x: x[1])
    
    split_dict = {}
    current_index = 0

    for keyword, position in keyword_positions:
        if current_index == 0:
            split_dict['name + flavor description'] = text[current_index:position].strip()
        elif current_index != 0:
            last_keyword, _ = keyword_positions[keyword_positions.index((keyword, position))-1]
            split_dict[last_keyword] = text[current_index:position].strip()
        current_index = position + len(keyword)

    if current_index != 0:
        last_keyword, _ = keyword_positions[-1]
        split_dict[last_keyword] = text[current_index:].strip()

    value = split_dict['name + flavor description']
    name = re.search("[a-zA-Z ]+[a-zA-Z]", value).group()

    # name 부분 텍스트 찾기
    name = value[:value.index(name) + len(name)].strip()
    # 영어 부분 찾기 (공백 포함)
    en_name = re.search("[a-zA-Z0-9 ]+[a-zA-Z]", value).group()
    # 영어 부분 앞의 텍스트 찾기 (한글 부분)
    ko_name = value[:value.index(en_name)]
    # flavor_description 부분 텍스트 찾기
    flavor_description = value[value.index(name) + len(name):]

    # 새로운 딕셔너리에 저장
    split_dict['name'] = ko_name.strip()
    split_dict['en_name'] = en_name.strip()
    split_dict['flavor description'] = flavor_description.strip()


    value = split_dict['Product Info']
    # 피쳐 키워드로 나누기
    feature_keywords = ['생산자', '국가/지역', '스타일', '용량', '기타(수상내역)', '품종']
    for word in feature_keywords:
        value = value.replace(word, "," + word + "," )
    value = value.replace(" ,", ",")
    value = value.replace(", ", ",")

    # 처음에 생기는 쉼표 제거
    value = value[1:] if value.startswith(",") else value
    split_dict['Product Info'] = value
    
    
    del split_dict['name + flavor description']
    try:
        del split_dict['Tasting Notes']
    except:
        pass
    try:
        split_dict['pairing'] = split_dict['Food Pairings']
        del split_dict['Food Pairings']
    except:
        pass
    try:
        split_dict['product info'] = split_dict['Product Info']
        del split_dict['Product Info']
    except:
        pass
    return split_dict

In [80]:
keywords = ['Product Info', 'Tasting Notes', 'Food Pairings']
raw_ocr_file_root_path = './data/wine_and_more/raw_ocr_results/'
ocr_file_root_path = './data/wine_and_more/ocr_results/'
os.makedirs(ocr_file_root_path, exist_ok=True)
raw_ocr_file_path_list = os.listdir(raw_ocr_file_root_path)

for raw_ocr_file_path in tqdm(raw_ocr_file_path_list):
    filename = os.path.basename(raw_ocr_file_path)
    with open(os.path.join(raw_ocr_file_root_path, raw_ocr_file_path), 'r', encoding='utf-8') as f:
        dictionary = json.load(f)
        unified_text = ''
        for text in dictionary['images'][0]['fields']:
            unified_text += text['inferText'] + ' '
        split_dict = split_text_to_dict(unified_text, keywords)

    with open(os.path.join(ocr_file_root_path, filename), 'w', encoding='utf-8') as f:
        json.dump(split_dict, f, ensure_ascii=False, indent=4)

100%|██████████| 549/549 [00:05<00:00, 99.27it/s] 


이제 ocr결과와 사이트에서 얻은 (./data/wine_and_more/wine_and_more.json)파일을 합쳐서  [wine and more 파일](./data/wine_and_more.json)로 만든다.

또 이를 winenara가 담겨있는 [unified wine 파일](./data/unified_wine_data.json)로 합친다. 

In [46]:
raw_data_list = []
with open('./data/wine_and_more/wine_and_more.json', 'r', encoding='utf-8') as f:
    while True:
        line = f.readline()
        if not line:
            break
        raw_data_list.append(json.loads(line))

In [39]:
raw_data_list[0]

{'url': 'https://www.wineandmore.co.kr/goods/goods_view.php?goodsNo=1000002574',
 'name': '투핸즈 아레스 매그넘',
 'price': '600,000',
 'feature_img_url': 'https://shinsegaelnb.cdn-nhncommerce.com/data/editor/goods/230614/90928309890675335d226e4a2bfd369d_134437.jpg',
 'img_url': 'https://shinsegaelnb.cdn-nhncommerce.com/data/goods/23/06/24/1000002574/1000002574_detail_047.jpg',
 'pickup_info': ['와인앤모어 합정역점']}

In [47]:
wine_and_more_attribute = ['price', 'name', 'url', 'img_url', 'pickup_info']
unified_attribute = ['url', 'site_name', 'price', 'name', 'en_name', 'img_url', 'body', 'acidity', 'tannin', "sweetness", "alcohol", "wine_type", 'country', 'grape', 'rating', 'pickup_location', 'vivino_link', 'flavor_description', 'pairing']
unified_dict = {'url':'url', 'site_name':'', 'price':'price', 'name':'name', 'en_name':'', 'img_url':'img_url', 'body':'', 'acidity':'', 'tannin':'', "sweetness":'', "alcohol":'', "wine_type":'', 'country':'', 'grape':'', 'rating':'', 'pickup_location':'pickup_info', 'vivino_link':'', 'flavor_description':'', 'pairing':''}
data_list = []
for data in raw_data_list:
    single_data = {}
    for unified_attr, wine_and_more_attribute in unified_dict.items():
        if wine_and_more_attribute != '':
            if type(data[wine_and_more_attribute]) == list:
                single_data[unified_attr] = ','.join(data[wine_and_more_attribute])
            else:
                single_data[unified_attr] = data[wine_and_more_attribute]          
        elif unified_attr == 'site_name':
            single_data[unified_attr] = 'wine_and_more'
        else:
            single_data[unified_attr] = ''
    data_list.append(single_data)

In [48]:
data_list[0]

{'url': 'https://www.wineandmore.co.kr/goods/goods_view.php?goodsNo=1000002574',
 'site_name': 'wine_and_more',
 'price': '600,000',
 'name': '투핸즈 아레스 매그넘',
 'en_name': '',
 'img_url': 'https://shinsegaelnb.cdn-nhncommerce.com/data/goods/23/06/24/1000002574/1000002574_detail_047.jpg',
 'body': '',
 'acidity': '',
 'tannin': '',
 'sweetness': '',
 'alcohol': '',
 'wine_type': '',
 'country': '',
 'grape': '',
 'rating': '',
 'pickup_location': '와인앤모어 합정역점',
 'vivino_link': '',
 'flavor_description': '',
 'pairing': ''}

이제 ocr을 통해 얻은 데이터(./data/wine_and_more/ocr_results)도 data_list에 추가해주도록 하자.

In [49]:
ocr_file_root_path = './data/wine_and_more/ocr_results/'
ocr_file_list = os.listdir(ocr_file_root_path)

for data in data_list:
    if data['name']+'.json' in ocr_file_list:
        with open(os.path.join(ocr_file_root_path, data['name']+'.json'), encoding='utf-8') as f:
            ocr_result = json.load(f)
        data['flavor_description'] = ocr_result['flavor description'] if 'flavor description' in ocr_result.keys() else ''
        data['pairing'] = ocr_result['pairing'] if 'pairing' in ocr_result.keys() else ''
        data['en_name'] = ocr_result['en_name'] if 'en_name' in ocr_result.keys() else ''

In [50]:
data_list[0]

{'url': 'https://www.wineandmore.co.kr/goods/goods_view.php?goodsNo=1000002574',
 'site_name': 'wine_and_more',
 'price': '600,000',
 'name': '투핸즈 아레스 매그넘',
 'en_name': 'Two Hands Ares',
 'img_url': 'https://shinsegaelnb.cdn-nhncommerce.com/data/goods/23/06/24/1000002574/1000002574_detail_047.jpg',
 'body': '',
 'acidity': '',
 'tannin': '',
 'sweetness': '',
 'alcohol': '',
 'wine_type': '',
 'country': '',
 'grape': '',
 'rating': '',
 'pickup_location': '와인앤모어 합정역점',
 'vivino_link': '',
 'flavor_description': "와이너리의 플래그쉽 와인으로 '전쟁의 신'을 뜻함 오너와 와인메이커가 모든 배럴을 시음해본 후 A+ 등급을 매긴 것만을 모아 만든 플래그쉽 와인으로, 바로사 밸리 '최상의 쉬라즈 와인 스타일을' 보여준다. 블랙베리, 향신료, 바닐라, 흰 꽃 등 다양한 아로마를 갖고 있으며 입 안에서 검붉은 과일의 집중된 맛을 느낄 수 있다. 은은하게 이어지는 라벤더, 파이프 담배, 바닐라 여운이 와인에 깊이감과 복합미를 더해준다. 잔에 따라두고 시간이 지나면 다크 초콜릿, 말린 후추의 알싸한 향도 느껴진다. 마세라시옹 과정을 통해 즙을 얻은 것, 이후에 다시 압착한 즙을 함께 사용해 양조한다.",
 'pairing': '양갈비구이, 스테이크 등 풍미가 진한 육류요리'}

In [51]:
with open('./data/wine_and_more.json', 'w', encoding='utf-8') as json_file:
    for data in data_list:
        json_file.write(json.dumps(data, ensure_ascii=False) + '\n')

In [52]:
with open('./data/unified_wine_data.json', 'a', encoding='utf-8') as json_file:
    for data in data_list:
        json_file.write(json.dumps(data, ensure_ascii=False) + '\n')

## CU 와인 크롤링

PocketCU라는 CU편의점 재고를 확인하는 앱이있다. 다행히 이 앱을 웹에서도 확인할 수 있다. 크게 아래의 3가지 url로 확인가능하다.
- 지도에서 아이템 재고 확인: [https://www.pocketcu.co.kr/search/stock?isRecommend=Y&item_cd={아이템id}](https://www.pocketcu.co.kr/search/stock?isRecommend=Y&item_cd=5011007015534)

<img src="./assets/cu_map.png" width=250 height=350>

- 아이템이름으로 검색: [https://www.pocketcu.co.kr/search/total/product/cubar?searchWord={아이템이름}](https://www.pocketcu.co.kr/search/total/product/cubar?searchWord=%EC%9C%84%EC%8A%A4%ED%82%A4)

<img src="./assets/cu_itemlist.png" width=250 height=350>

- 특정 매장에 어떤 아이템의 재고 확인: [https://www.pocketcu.co.kr/search/stock/storeItems/{점포id}?searchWord={아이템이름}](https://www.pocketcu.co.kr/search/stock/storeItems/04456?searchWord=%EC%A0%9C%EC%9E%84%EC%8A%A8)

<img src="./assets/cu_detailpage.png" width=250 height=350>

### Vivino에서 와인 정보 가져오기

Vivino에서 api를 사용해 정보를 가져온다.

In [3]:
import requests


class Requester:
    """Wraps basic `requests` package functions into a class.

    """

    def __init__(self, base_url):
        """Initializition method.

        Args:
            base_url (str): String that defines the base URL.

        """

        # Defines the base URL to be used
        self.base_url = base_url

        # Defines the default headers to be used
        self.headers = {
            "User-Agent": ""
        }

    def get(self, endpoint, **kwargs):
        """Wraps the `get` method of `requests`.

        Args:
            endpoint (str): Endpoint to be consumed.

        Returns:
            The `get` function with pre-populated headers over the endpoint.

        """

        # Defines the full-path URL
        url = self.base_url + endpoint

        return requests.get(url, headers=self.headers, **kwargs)


In [1]:
country_code_list = ['af', 'ax', 'al', 'dz', 'as', 'ad', 'ao', 'ai', 'aq', 'ag', 'ar', 'am', 'aw', 'au', 'at', 'az', 'bs', 'bh', 'bd', 'bb', 'by', 'be', 'bz', 'bj', 'bm', 'bt', 'bo', 'ba', 'bw', 'br', 'io', 'bn', 'bg', 'bf', 'bi', 'kh', 'cm', 'ca', 'cv', 'ky', 'cf', 'td', 'cl', 'cn', 'cx', 'cc', 'co', 'km', 'cg', 'cd', 'ck', 'cr', 'ci', 'hr', 'cu', 'cy', 'cz', 'dk', 'dj', 'dm', 'do', 'ec', 'eg', 'sv', 'gq', 'er', 'ee', 'et', 'fk', 'fo', 'fj', 'fi', 'fr', 'gf', 'pf', 'ga', 'gm', 'ge', 'de', 'gh', 'gi', 'gr', 'gl', 'gd', 'gp', 'gu', 'gt', 'gg', 'gn', 'gw', 'gy', 'ht', 'va', 'hn', 'hk', 'hu', 'is', 'in', 'id', 'ir', 'iq', 'ie', 'im', 'il', 'it', 'jm', 'jp', 'je', 'jo', 'kz', 'ke', 'ki', 'kp', 'kr', 'kw', 'kg', 'la', 'lv', 'lb', 'ls', 'lr', 'ly', 'li', 'lt', 'lu', 'mo', 'mk', 'mg', 'mw', 'my', 'mv', 'ml', 'mt', 'mh', 'mq', 'mr', 'mu', 'yt', 'mx', 'fm', 'md', 'mc', 'mn', 'me', 'ms', 'ma', 'mz', 'mm', 'na', 'nr', 'np', 'nl', 'an', 'nc', 'nz', 'ni', 'ne', 'ng', 'nu', 'nf', 'mp', 'no', 'om', 'pk', 'pw', 'ps', 'pa', 'pg', 'py', 'pe', 'ph', 'pn', 'pl', 'pt', 'pr', 'qa', 'ro', 'ru', 'rw', 're', 'bl', 'sh', 'kn', 'lc', 'mf', 'pm', 'vc', 'ws', 'sm', 'st', 'sa', 'sn', 'rs', 'sc', 'sl', 'sg', 'sk', 'si', 'sb', 'so', 'za', 'ss', 'gs', 'es', 'lk', 'sd', 'sr', 'sj', 'sz', 'se', 'ch', 'sy', 'tw', 'tj', 'tz', 'th', 'tl', 'tg', 'tk', 'to', 'tt', 'tn', 'tr', 'tm', 'tc', 'tv', 'ug', 'ua', 'ae', 'gb', 'us', 'uy', 'uz', 'vu', 've', 'vn', 'vg', 'vi', 'wf', 'ye', 'zm', 'zw']

In [7]:
import argparse
import json

BASE_URL = 'https://www.vivino.com/api/'

# Number of records per page
RECORDS_PER_PAGE = 25

output_file = 'vivino.json'
start_page = 1

# Instantiates a wrapper over the `requests` package
r = Requester(BASE_URL)

# Defines the payload, i.e., filters to be used on the search
for country_code in country_code_list[56:]:
    payload = {
        "country_code": country_code,
        # "country_codes[]": "AU",
        # "food_ids[]": 20,
        # "grape_ids[]": 3,
        # "grape_filter": "varietal",
        # "max_rating": 5.0,
        "min_rating": 2.0,
        # "order_by": "ratings_average",
        # "order": "desc",
        # "price_range_min": 0,
        # "price_range_max": 100000000000000,
        # "region_ids[]": 383,
        # "wine_style_ids[]": 98,
        # "wine_type_ids[]": 1,
        # "wine_type_ids[]": 2,
        # "wine_type_ids[]": 3,
        # "wine_type_ids[]": 4,
        # "wine_type_ids[]": 7,
        # "wine_type_ids[]": 24,
    }

    # Performs an initial request to get the number of records (wines)
    res = r.get('explore/explore?', params=payload)
    n_matches = res.json()['explore_vintage']['records_matched']

    print(country_code, f'Number of matches: {n_matches}')

    # Iterates through the amount of possible pages
    for i in range(start_page, max(1, int(n_matches / RECORDS_PER_PAGE)) + 1):
    # for i in range(start_page, 2):
        # Adds the page to the payload
        payload['page'] = i

        print(f'Scraping data from page: {payload["page"]}')

        # Performs the request and scraps the URLs
        res = r.get('explore/explore', params=payload)
        try:
            matches = res.json()['explore_vintage']['matches']
        except:
            continue

        # Opens the output file with append
        with open(output_file, 'a') as f:
            # Iterates over every match
            for match in matches:
                wine_data = match['vintage']['wine']
                try:
                    wine_name = wine_data['name']
                except:
                    wine_name = ''
                
                try:
                    wine_id = wine_data['id']
                except:
                    wine_id = -1

                try:
                    wine_type_id = wine_data['type_id']
                except:
                    wine_type_id = -1

                try:
                    wine_region_name = wine_data['region']['name']
                except:
                    wine_region_name = ''

                try:
                    wine_region_country_name = wine_data['region']['country']['name']
                except:
                    wine_region_country_name = ''

                try:
                    wine_grapes_name = '||'.join([grape['name'] for grape in wine_data['region']['country']['most_used_grapes']])
                except:
                    wine_grapes_name = ''

                try:
                    wine_winery_name = wine_data['winery']['name']
                except:
                    wine_winery_name = ''

                wine_flavor_primary_list = []
                try:
                    wine_taste_structure = wine_data['taste']['structure']
                    wine_acidity = wine_taste_structure['acidity'] if 'acidity' in wine_taste_structure else -1
                    wine_fizziness = wine_taste_structure['fizziness'] if 'fizziness' in wine_taste_structure else -1
                    wine_intensity = wine_taste_structure['intensity'] if 'intensity' in wine_taste_structure else -1
                    wine_sweetness = wine_taste_structure['sweetness'] if 'sweetness' in wine_taste_structure else -1
                    wine_tannin = wine_taste_structure['tannin'] if 'tannin' in wine_taste_structure else -1
                    wine_flavor_primary_list.extend([flavor['primary_keywords'] for flavor in  wine_data['taste']['flavor']])
                    top_5_flavors = sorted(wine_flavor_primary_list[0], key=lambda x: x['count'], reverse=True)[:5]
                    top_5_flavors = '||'.join([flavor['name'] for flavor in top_5_flavors])
                except:
                    wine_acidity = -1
                    wine_fizziness = -1
                    wine_intensity = -1
                    wine_sweetness = -1
                    wine_tannin = -1
                    top_5_flavors = ''

                
                try:
                    wine_food_names = '||'.join([food['name'] for food in wine_data['style']['food']])
                except:
                    wine_food_names = ''

                # 추출된 속성을 dictionary로 저장
                wine_dict = {
                    'name': wine_name,
                    'id': wine_id,
                    'type_id': wine_type_id,
                    'region_name': wine_region_name,
                    'region_country_name': wine_region_country_name,
                    'grapes_name': wine_grapes_name,
                    'winery_name': wine_winery_name,
                    'acidity': wine_acidity if wine_acidity != None else -1,
                    'fizziness': wine_fizziness if wine_fizziness != None else -1,
                    'intensity': wine_intensity if wine_intensity != None else -1,
                    'sweetness': wine_sweetness if wine_sweetness != None else -1,
                    'tannin': wine_tannin if wine_tannin != None else -1,
                    'flavors': top_5_flavors,
                    'food_pairing': wine_food_names,
                }

                # dictionary를 JSON 형식으로 파일에 저장
                json_data = json.dumps(wine_dict)
                f.write(f'{json_data}\n')

cz Number of matches: 32919
Scraping data from page: 1
Scraping data from page: 2
Scraping data from page: 3
Scraping data from page: 4
Scraping data from page: 5
Scraping data from page: 6
Scraping data from page: 7
Scraping data from page: 8
Scraping data from page: 9
Scraping data from page: 10
Scraping data from page: 11
Scraping data from page: 12
Scraping data from page: 13
Scraping data from page: 14
Scraping data from page: 15
Scraping data from page: 16
Scraping data from page: 17
Scraping data from page: 18
Scraping data from page: 19
Scraping data from page: 20
Scraping data from page: 21
Scraping data from page: 22
Scraping data from page: 23
Scraping data from page: 24
Scraping data from page: 25
Scraping data from page: 26
Scraping data from page: 27
Scraping data from page: 28
Scraping data from page: 29
Scraping data from page: 30
Scraping data from page: 31
Scraping data from page: 32
Scraping data from page: 33
Scraping data from page: 34
Scraping data from page: 35
S

In [6]:
country_code_list.index('cz')

56

#### 수집된 vivino 중 중복 제거하기

In [8]:
import pandas as pd

df = pd.read_json('vivino.json', lines=True)
df.drop_duplicates(subset='name', keep='first', inplace=True)
df.to_json('vivino_processed.json', orient='records', lines=True)

### 이름으로 한국의 와인몰과 vivino 연결

In [ ]:
import pandas as pd

In [112]:
vivino_df = pd.read_json('vivino.json', lines=True)

In [113]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [114]:
vivino_df['name']

0                                 Essencia
1                                Sassicaia
2              Pauillac (Grand Cru Classé)
3                              Clos Apalta
4      Pauillac (Premier Grand Cru Classé)
                      ...                 
960                             Hey Malbec
961                    Expression Bordeaux
962              Tramari Rosé di Primitivo
963         Bourbon Barrels Aged Red blend
964          'Rippon' Mature Vine Riesling
Name: name, Length: 965, dtype: object

In [118]:
wine_and_more_wine = 'Domaine Geantet Pansiot Gevrey Chambertin En Champs'

In [121]:
for wine in vivino_df['name']:
    if similar(wine_and_more_wine, wine) > 0.4:
        print(wine)

Premiere Napa Valley Cabernet Sauvignon
Gevrey-Chambertin
Grand Reserve Chardonnay
